In [ ]:
import multiprocessing
import time
from process import process
from process import plotPolarColor
import datetime
import os
import socket
import pickle
import sys
import time

import pyqtgraph as pg
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QWidget
from PyQt5.QtCore import Qt, QTimer
import csiread
import numpy as np

import matplotlib.pyplot as plt
import struct
import sys
import time


    
def process1(queue1):
    # 第一个进程，每1秒向队列中放入一个值
    SIZE_BUFFER = 3600
    HZ = 800
    TIME = 20000
    NUM_TOTAL = HZ * TIME

    socket.NETLINK_CONNECTOR = 11
    CN_NETLINK_USERS = 11
    CN_IDX_IWLAGN = CN_NETLINK_USERS + 0xF
    NETLINK_ADD_MEMBERSHIP = 1


    # --------------------------------- realtime --------------------------------- #


    L = 40  # fixed window length
    FPS_RATIO = 2  # eval per second
    FRAC = int(L / FPS_RATIO)

    Ntx = 1
    Nrx = 3
    Nsub = 30

    csi_L = np.zeros((L, Nsub, Nrx, Ntx))
    csi_frac = np.zeros((FRAC, Nsub, Nrx, Ntx))
    
    while True:

        csidata = csiread.Intel(None, Nrx, Ntx)
        count = 0

        with socket.socket(
            socket.AF_NETLINK, socket.SOCK_DGRAM, socket.NETLINK_CONNECTOR
        ) as s:
            s.bind((os.getpid(), CN_IDX_IWLAGN))
            s.setsockopt(270, NETLINK_ADD_MEMBERSHIP, CN_IDX_IWLAGN)

            while count < NUM_TOTAL:
                ret = s.recv(SIZE_BUFFER)
                # keep nothing but data part
                cnmsg_data = ret[36:]
                # parse data using csiread.Intel.pmsg
                # only requires data bytes, omitting any other bytes
                csi_i = None
                status = csidata.pmsg(cnmsg_data)
                # csi from one packet
                if status == 0xBB:
                    # 187, status valid, else error
                    csi_i = csidata.get_scaled_csi()
                    count += 1
                    queue1.put(csi_i)

                   
                        

            s.close()

def process2(queue2):
    SIZE_BUFFER = 1440
    HZ = 800


    socket.NETLINK_CONNECTOR = 11
    CN_NETLINK_USERS = 11
    CN_IDX_IWLAGN = CN_NETLINK_USERS + 0xF
    NETLINK_ADD_MEMBERSHIP = 1

        
    address_src = ("10.20.14.28", 10086)
    address_des = ("10.20.14.37", 10010)
    # 创建socket对象，并绑定IP地址和端口号
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        s.bind(address_des)
        while True:
            csi_L,address_src = s.recvfrom(SIZE_BUFFER)
            csi = np.frombuffer(csi_L,dtype=np.complex128).reshape((1,30,3,1))
            #csi = np.frombuffer(csi_L,dtype=np.complex128)
            queue2.put(csi)
            #print(csi[0,28,0,0])
            #csi_reshape = matrixreshape(csi)
            #print(csi_reshape[28,0])
        s.close()
            
def matrixreshape(original_array):
    length = original_array.shape[0]
    # 新数组形状为40x90
    reshaped_array = np.zeros((length, 90), dtype=np.complex128)

    # 将第三维的数据合并到新数组中
    for i in range(3):
        start_idx = i * 30
        end_idx = (i + 1) * 30
        reshaped_array[:, start_idx:end_idx] = original_array[:, :, i, 0]
    
    return reshaped_array.T

class RealTimePlotWidget(QWidget):
    def __init__(self):
        super().__init__()

        self.complex_data1 = []  # 第一个复数数据数组
        self.complex_data2 = []  # 第二个复数数据数组
        self.complex_data3 = []  # 第三个复数数据数组

        self.plot1 = pg.PlotWidget(title="Plot 1")  # 设置第一个图形的标题
        self.plot2 = pg.PlotWidget(title="Plot 2")  # 设置第二个图形的标题
        self.plot3 = pg.PlotWidget(title="Plot 3")  # 设置第三个图形的标题

        layout = QVBoxLayout()
        layout.addWidget(self.plot1)
        layout.addWidget(self.plot2)
        layout.addWidget(self.plot3)
        self.setLayout(layout)

        self.plot1.setXRange(0, 1)  # 设置第一个图形的x轴范围
        self.plot1.setYRange(0, 1)  # 设置第一个图形的y轴范围
        self.plot1.setLabel('left', 'Imaginary')  # 设置第一个图形的y轴标签
        self.plot1.setLabel('bottom', 'Real')  # 设置第一个图形的x轴标签

        self.plot2.setXRange(0, 1)  # 设置第二个图形的x轴范围
        self.plot2.setYRange(0, 1)  # 设置第二个图形的y轴范围
        self.plot2.setLabel('left', 'Imaginary')  # 设置第二个图形的y轴标签
        self.plot2.setLabel('bottom', 'Real')  # 设置第二个图形的x轴标签

        self.plot3.setXRange(0, 1)  # 设置第三个图形的x轴范围
        self.plot3.setYRange(0, 1)  # 设置第三个图形的y轴范围
        self.plot3.setLabel('left', 'Imaginary')  # 设置第三个图形的y轴标签
        self.plot3.setLabel('bottom', 'Real')  # 设置第三个图形的x轴标签

        self.timer = QTimer()
        self.timer.timeout.connect(self.update_plot)
        self.timer.start(100)  # 每100毫秒更新一次图形

    def update_plot(self,loc,ne1,ne2):

        self.complex_data1.append(loc)  # 将新数据添加到第一个数组中
        self.complex_data2.append(ne1)  # 将新数据添加到第二个数组中
        self.complex_data3.append(ne2)  # 将新数据添加到第三个数组中

        self.plot1.clear()  # 清除第一个图形
        self.plot2.clear()  # 清除第二个图形
        self.plot3.clear()  # 清除第三个图形

        x1 = np.real(self.complex_data1)  # 第一个数组的实部作为 x 轴
        y1 = np.imag(self.complex_data1)  # 第一个数组的虚部作为 y 轴
        color1 = np.linspace(0, 1, len(self.complex_data1))  # 创建颜色渐变序列
        # 绘制第一个数组的数据点，颜色从蓝色到红色渐变
        self.plot1.plot(x1, y1, pen=pg.mkPen(color=(255*color1[-1], 0, 255*(1-color1[-1]), 200)))

        x2 = np.real(self.complex_data2)  # 第二个数组的实部作为 x 轴
        y2 = np.imag(self.complex_data2)  # 第二个数组的虚部作为 y 轴
        # 绘制第二个数组的数据点，颜色为绿色
        self.plot2.plot(x2, y2, pen=pg.mkPen(color=(0, 255, 0, 200)))

        x3 = np.real(self.complex_data3)  # 第三个数组的实部作为 x 轴
        y3 = np.imag(self.complex_data3)  # 第三个数组的虚部作为 y 轴
        # 绘制第三个数组的数据点，颜色为红色
        self.plot3.plot(x3, y3, pen=pg.mkPen(color=(255, 255, 0, 200)))
if __name__ == '__main__':
    # 创建三个队列
    queue1 = multiprocessing.Queue()
    queue2 = multiprocessing.Queue()
    # 创建三个进程
    p1 = multiprocessing.Process(target=process1, args=(queue1,))
    p2 = multiprocessing.Process(target=process2, args=(queue2,))
    
    p1.start()
    p2.start()
       
    tx = 1 + 2j
    r1 = 0 + 0j

    # r2=3+0i
    r2 = 3 + 0j
    rx = [r1, r2]
    loc_ini = 0+1.5j
    
    app = QApplication(sys.argv)
    mainWindow = QMainWindow()
    plotWidget = RealTimePlotWidget()
    mainWindow.setCentralWidget(plotWidget)
    mainWindow.show()
    

    # 从队列3中取出值
    while True:
        loc=[]
        csi_1=[]
        csi_2=[]
        while len (csi_1)<2400:
            data1 = queue1.get()
            csi_data1 = matrixreshape(data1)
            csi_1.append(csi_data1)
            data2 =  queue2.get()
            csi_data2 = matrixreshape(data2)
            csi_2.append(csi_data2)
        csi_d1=np.concatenate(csi_1,axis=1)
        csi_d2=np.concatenate(csi_2,axis=1)
        #print(csi_d1.shape)

        print(loc_ini)
        [loc,ne1,ne2] = process(csi_d1,csi_d2,800,loc_ini)
        RealTimePlotWidget.update_plot(plotWidget,loc,ne1,ne2)
        
        loc_ini = loc[-1]
        import multiprocessing
import time
from process import process
from process import plotPolarColor
import datetime
import os
import socket
import pickle
import sys
import time

import pyqtgraph as pg
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QWidget
from PyQt5.QtCore import Qt, QTimer
import csiread
import numpy as np

import matplotlib.pyplot as plt
import struct
import sys
import time


    
def process1(queue1):
    # 第一个进程，每1秒向队列中放入一个值
    SIZE_BUFFER = 3600
    HZ = 800
    TIME = 20000
    NUM_TOTAL = HZ * TIME

    socket.NETLINK_CONNECTOR = 11
    CN_NETLINK_USERS = 11
    CN_IDX_IWLAGN = CN_NETLINK_USERS + 0xF
    NETLINK_ADD_MEMBERSHIP = 1


    # --------------------------------- realtime --------------------------------- #


    L = 40  # fixed window length
    FPS_RATIO = 2  # eval per second
    FRAC = int(L / FPS_RATIO)

    Ntx = 1
    Nrx = 3
    Nsub = 30

    csi_L = np.zeros((L, Nsub, Nrx, Ntx))
    csi_frac = np.zeros((FRAC, Nsub, Nrx, Ntx))
    
    while True:

        csidata = csiread.Intel(None, Nrx, Ntx)
        count = 0

        with socket.socket(
            socket.AF_NETLINK, socket.SOCK_DGRAM, socket.NETLINK_CONNECTOR
        ) as s:
            s.bind((os.getpid(), CN_IDX_IWLAGN))
            s.setsockopt(270, NETLINK_ADD_MEMBERSHIP, CN_IDX_IWLAGN)

            while count < NUM_TOTAL:
                ret = s.recv(SIZE_BUFFER)
                # keep nothing but data part
                cnmsg_data = ret[36:]
                # parse data using csiread.Intel.pmsg
                # only requires data bytes, omitting any other bytes
                csi_i = None
                status = csidata.pmsg(cnmsg_data)
                # csi from one packet
                if status == 0xBB:
                    # 187, status valid, else error
                    csi_i = csidata.get_scaled_csi()
                    count += 1
                    queue1.put(csi_i)

                   
                        

            s.close()

def process2(queue2):
    SIZE_BUFFER = 1440
    HZ = 800


    socket.NETLINK_CONNECTOR = 11
    CN_NETLINK_USERS = 11
    CN_IDX_IWLAGN = CN_NETLINK_USERS + 0xF
    NETLINK_ADD_MEMBERSHIP = 1

        
    address_src = ("10.20.14.28", 10086)
    address_des = ("10.20.14.37", 10010)
    # 创建socket对象，并绑定IP地址和端口号
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        s.bind(address_des)
        while True:
            csi_L,address_src = s.recvfrom(SIZE_BUFFER)
            csi = np.frombuffer(csi_L,dtype=np.complex128).reshape((1,30,3,1))
            #csi = np.frombuffer(csi_L,dtype=np.complex128)
            queue2.put(csi)
            #print(csi[0,28,0,0])
            #csi_reshape = matrixreshape(csi)
            #print(csi_reshape[28,0])
        s.close()
            
def matrixreshape(original_array):
    length = original_array.shape[0]
    # 新数组形状为40x90
    reshaped_array = np.zeros((length, 90), dtype=np.complex128)

    # 将第三维的数据合并到新数组中
    for i in range(3):
        start_idx = i * 30
        end_idx = (i + 1) * 30
        reshaped_array[:, start_idx:end_idx] = original_array[:, :, i, 0]
    
    return reshaped_array.T

class RealTimePlotWidget(QWidget):
    def __init__(self):
        super().__init__()

        self.complex_data1 = []  # 第一个复数数据数组
        self.complex_data2 = []  # 第二个复数数据数组
        self.complex_data3 = []  # 第三个复数数据数组

        self.plot1 = pg.PlotWidget(title="Plot 1")  # 设置第一个图形的标题
        self.plot2 = pg.PlotWidget(title="Plot 2")  # 设置第二个图形的标题
        self.plot3 = pg.PlotWidget(title="Plot 3")  # 设置第三个图形的标题

        layout = QVBoxLayout()
        layout.addWidget(self.plot1)
        layout.addWidget(self.plot2)
        layout.addWidget(self.plot3)
        self.setLayout(layout)

        self.plot1.setXRange(0, 1)  # 设置第一个图形的x轴范围
        self.plot1.setYRange(0, 1)  # 设置第一个图形的y轴范围
        self.plot1.setLabel('left', 'Imaginary')  # 设置第一个图形的y轴标签
        self.plot1.setLabel('bottom', 'Real')  # 设置第一个图形的x轴标签

        self.plot2.setXRange(0, 1)  # 设置第二个图形的x轴范围
        self.plot2.setYRange(0, 1)  # 设置第二个图形的y轴范围
        self.plot2.setLabel('left', 'Imaginary')  # 设置第二个图形的y轴标签
        self.plot2.setLabel('bottom', 'Real')  # 设置第二个图形的x轴标签

        self.plot3.setXRange(0, 1)  # 设置第三个图形的x轴范围
        self.plot3.setYRange(0, 1)  # 设置第三个图形的y轴范围
        self.plot3.setLabel('left', 'Imaginary')  # 设置第三个图形的y轴标签
        self.plot3.setLabel('bottom', 'Real')  # 设置第三个图形的x轴标签

        self.timer = QTimer()
        self.timer.timeout.connect(self.update_plot)
        self.timer.start(100)  # 每100毫秒更新一次图形

    def update_plot(self,loc,ne1,ne2):

        self.complex_data1.append(loc)  # 将新数据添加到第一个数组中
        self.complex_data2.append(ne1)  # 将新数据添加到第二个数组中
        self.complex_data3.append(ne2)  # 将新数据添加到第三个数组中

        self.plot1.clear()  # 清除第一个图形
        self.plot2.clear()  # 清除第二个图形
        self.plot3.clear()  # 清除第三个图形

        x1 = np.real(self.complex_data1)  # 第一个数组的实部作为 x 轴
        y1 = np.imag(self.complex_data1)  # 第一个数组的虚部作为 y 轴
        color1 = np.linspace(0, 1, len(self.complex_data1))  # 创建颜色渐变序列
        # 绘制第一个数组的数据点，颜色从蓝色到红色渐变
        self.plot1.plot(x1, y1, pen=pg.mkPen(color=(255*color1[-1], 0, 255*(1-color1[-1]), 200)))

        x2 = np.real(self.complex_data2)  # 第二个数组的实部作为 x 轴
        y2 = np.imag(self.complex_data2)  # 第二个数组的虚部作为 y 轴
        # 绘制第二个数组的数据点，颜色为绿色
        self.plot2.plot(x2, y2, pen=pg.mkPen(color=(0, 255, 0, 200)))

        x3 = np.real(self.complex_data3)  # 第三个数组的实部作为 x 轴
        y3 = np.imag(self.complex_data3)  # 第三个数组的虚部作为 y 轴
        # 绘制第三个数组的数据点，颜色为红色
        self.plot3.plot(x3, y3, pen=pg.mkPen(color=(255, 255, 0, 200)))
if __name__ == '__main__':
    # 创建三个队列
    queue1 = multiprocessing.Queue()
    queue2 = multiprocessing.Queue()
    # 创建三个进程
    p1 = multiprocessing.Process(target=process1, args=(queue1,))
    p2 = multiprocessing.Process(target=process2, args=(queue2,))
    
    p1.start()
    p2.start()
       
    tx = 1 + 2j
    r1 = 0 + 0j

    # r2=3+0i
    r2 = 3 + 0j
    rx = [r1, r2]
    loc_ini = 0+1.5j
    
    app = QApplication(sys.argv)
    mainWindow = QMainWindow()
    plotWidget = RealTimePlotWidget()
    mainWindow.setCentralWidget(plotWidget)
    mainWindow.show()
    

    # 从队列3中取出值
    while True:
        loc=[]
        csi_1=[]
        csi_2=[]
        while len (csi_1)<2400:
            data1 = queue1.get()
            csi_data1 = matrixreshape(data1)
            csi_1.append(csi_data1)
            data2 =  queue2.get()
            csi_data2 = matrixreshape(data2)
            csi_2.append(csi_data2)
        csi_d1=np.concatenate(csi_1,axis=1)
        csi_d2=np.concatenate(csi_2,axis=1)
        #print(csi_d1.shape)

        print(loc_ini)
        [loc,ne1,ne2] = process(csi_d1,csi_d2,800,loc_ini)
        RealTimePlotWidget.update_plot(plotWidget,loc,ne1,ne2)
        
        loc_ini = loc[-1]
        sys.exit(app.exec_())
    

: 

: 